# STL to block mesh dictionary python script 
### 12/26/22
### update 1/4/23

In [2]:
from __future__ import unicode_literals, print_function

import math
import numpy as np
from ofblockmeshdicthelper import BlockMeshDict, Vertex, SimpleGrading


In [4]:
# take in box parameters and build a mesh around them 
box_params = { #m
    "x": 4.65, # x-z plane faces front of the wind tunnel, width 
    "y": 3.65, # depth of the box 
    "z": 3.66 # height 0 vertical 
}

## from block mesh dict 

### wind tunnel 

In [7]:
# ramponi + blocken 2012        
# * H_D = H + d, 
# * W_D = W + 2d
# * H, W = height and width of the building respectively 
# * d = 5H - 3H (start with 3H for early study) 
# * Upstream, downstream length => 5H and 15H 

H = box_params["z"]
d = 3 * H # 3H 

wt_params = {
    "x": box_params["x"] + 2*d,   # width 
    "y": box_params["y"] + 5*H + 15*H, # depth 
    "z": box_params["z"] + d, # height 
}
wt_params


{'x': 26.61, 'y': 76.85000000000001, 'z': 14.64}

In [ ]:
# geometries
# wt_length = 20 # wt  = wind tunnel x-dir
# wt_depth = 3 # z-dir
# wt_height = 3 # y-dir
# box_size = 1

In [ ]:
# prepare ofblockmeshdicthelper.BlockMeshDict instance to
# gather vertices, blocks, faces and boundaries.
bmd = BlockMeshDict()

# set metrics
bmd.set_metric('m')


In [ ]:

# base vertices of wind tunnel 
basevs = [
    Vertex(0, 0, 0, 'v0'),
    Vertex(wt_length, 0, 0, 'v1'),
    Vertex(wt_length,  0, wt_depth, 'v2'),
    Vertex(0, 0, wt_depth, 'v3')]

for v in basevs:
    print(f"{v.name}: {v.x, v.y, v.z}")

In [ ]:
def vnamegen(x0z0, x1z0, x1z1, x0z1):
    return (x0z0+'-y', x1z0+'-y', x1z0+'+y', x0z0+'+y',
            x0z1+'-y', x1z1+'-y', x1z1+'+y', x0z1+'+y')

In [ ]:
b0 = bmd.add_hexblock(vnamegen('v0', 'v1', 'v2', 'v3'),
                      (80, 50, 20),
                      'b0',
                      grading=SimpleGrading(1,
                                            1,
                                            1))

In [ ]:
bmd.add_boundary('wall', 'bottom', [b0.face('s')]) ## should probably be patches?
bmd.add_boundary('patch', 'top', [b0.face('n')])

bmd.add_boundary('patch', 'outlet', [b0.face('e')])
bmd.add_boundary('patch', 'inlet', [b0.face('w')])

bmd.add_boundary('symmetry', 'side01', [b0.face('b')])
bmd.add_boundary('symmetry', 'side02', [b0.face('t')])

### internal block 

In [ ]:
# now want internal block
# base vertices of wind tunnel 
boxvs = [
    Vertex(box_size, 0, box_size, 'v4'),
    Vertex(box_size + box_size, 0, box_size, 'v5'),
    Vertex(box_size + box_size,  0, box_size + box_size, 'v6'),
    Vertex(box_size, 0, box_size + box_size, 'v7')]

In [ ]:
# raise up in the y direction 
for v in boxvs:
    bmd.add_vertex(v.x, box_size, v.z, v.name+'+y')
    bmd.add_vertex(v.x, 0, v.z, v.name+'-y')

In [ ]:
b1 = bmd.add_hexblock(vnamegen('v4', 'v5', 'v6', 'v7'),
                      (2, 5, 5),
                      'b1',
                      grading=SimpleGrading(1,
                                            1,
                                            1))
[b1.face(i).name for i in range(6)]

In [ ]:
bmd.add_boundary('wall', 'box', [b1.face(i) for i in range(6) ]) 

# Finish 

In [ ]:
bmd.assign_vertexid()

In [ ]:
print(bmd.format())